## Toy Piece

Start the audio server.

In [1]:
ServerOptions.devices

In [2]:
s.options.outDevice_("Out_Soundflower_Built-In"); // An aggregate device to capture audio during screen recording

In [3]:
s.boot;

In [4]:
s.meter;

In [5]:
FreqScope.new

## Instruments

Tone generated by sending an impulse through resonant filters.

In [6]:
SynthDef('tone', {
    arg outBus = 0, freq = 521, amp = 0.2, pos = 0, atk = 0.05;
    var sig;

    sig = Klank.ar(`[
        [freq, freq*1.501, freq*1.97], // freqs
        [0.7, 0.45, 0.25], // amps
        [0.2, 0.3, 0.48] // phases
    ], Impulse.ar(0));
    DetectSilence.ar(sig, doneAction: 2);
    sig = sig * EnvGen.kr(Env([0, 1], [atk]));
    sig = Pan2.ar(sig, pos, amp);

    Out.ar(outBus, sig);
}).add;

String like sound generated by using a variable triangle/saw wave.

In [7]:
SynthDef('bleep', {
    arg outBus = 0, freq = 440, amp = 0.2, len = 1, pos = 0;
    var sig, env;

    env = EnvGen.kr(
        Env([0, amp, 0], [len * (2/3), len/3], curve: 'cubed'),
        doneAction: 2
    );

    sig = VarSaw.ar(
        freq: freq,
        iphase: 0,
        width: env,
        mul: 1
    );
    sig = sig * env;
    sig = Pan2.ar(sig, pos);

    Out.ar(outBus, sig);
}).add;

## Reverbs

A reverberator created through a chain of series allpass filters with dampening from a lowpass filter.  `mix` parameter controls the amount of reverb.

In [8]:
SynthDef('reverb', {
    arg outBus = 0, inBus, mix = 0.4;
    var dry, wet, sig, numFilters = 8;

    dry = In.ar(inBus, 2);

    wet = dry;
    numFilters.do({
        wet = AllpassN.ar(
            in: wet,
            maxdelaytime: 0.2,
            delaytime: {Rand(0.05, 0.15)} ! 2,
            decaytime: {Rand(1, 2)} ! 2
        );
    });

    wet = LPF.ar(wet, 3500);
    sig = dry.blend(wet, mix);

    Out.ar(outBus, sig);
}).add;

A very wet, washy reverb intended to create background drone.

In [9]:
SynthDef('wash', {
    arg outBus = 0, inBus;
    var sig, numFilters = 16;

    sig = In.ar(inBus, 2);

    numFilters.do({
        sig = AllpassN.ar(
            in: sig,
            maxdelaytime: 0.2,
            delaytime: {Rand(0.05, 0.15)} ! 2,
            decaytime: {Rand(2, 4)} ! 2
        );
    });

    sig = LPF.ar(sig, 3500);

    Out.ar(outBus, sig);
}).add;

## Setup

Establish a tempo and instantiate both reverbs.

In [10]:
~tempo = 60;
~clock = TempoClock(~tempo/60);
~washBus = Bus.audio(s, 2);
~washSynth = Synth('wash', [\inBus, ~washBus]);
~reverbBus = Bus.audio(s, 2);
~reverbSynth = Synth('reverb', [\inBus, ~reverbBus]);

## Workspace

In [17]:
Pbindef('tonePattern',
    'instrument', 'tone',
    'scale', #[0, 2, 3, 5, 7, 8, 10],
    'degree', Prand([0, 1, 2, 3, 4, 5, 6], inf),
    'amp', Pwhite(0.2, 0.3),
    'dur', Pwhite(0.1, 0.2),
    'octave', 6,
    'atk', 0.004,
    'outBus', ~washBus
).play(~clock);

In [28]:
Pbindef('tonePattern').stop;

In [14]:
Pbindef('string',
    'instrument', 'bleep',
    'scale', #[0, 2, 3, 5, 7, 8, 10],
    'degree', Prand([0, 2, 3], inf),
    'amp', 0.2,
    'octave', 4,
    'len', Pwhite(1, 3),
    'dur', 3,
    'pos', Pwhite(-0.5, 0.5),
    'outBus', ~washBus
).play(~clock);

In [27]:
Pbindef('string').stop;

## Cleanup

In [29]:
Pbindef.clear;